In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun  9 17:09:58 2019

@author: chris
"""

# This file contains the names and URL of companies to scrape
# Start: emtpy file
# End: companies in SQLite database

# Step1D: Additional characteristics can be added to the company info

# code taken from http://www.sqlitetutorial.net/sqlite-python/creating-database/
import sqlite3
from scraper import create_connection, create_table, modify_table, read_CSV, write_to_db, retrieve_data, write_to_db_scrape1, scrape
import bs4

database = "scrape.db"


# Step 1: Setup database

## Step1A: Set up a SQLite database

In [ ]:
def create_database():
        # define the database        
        global database                
        # create a database connection
        conn=create_connection(database)
        conn.close()
        print("done")

create_database()

    

## Step1B: Set up tables in the database

In [ ]:
sql_create_companies_table = """
        CREATE TABLE IF NOT EXISTS companies(
        id integer PRIMARY KEY,
        Company text NOT NULL,
        Website text,
        CountryOfIncorporation text,
        MarketDomain text        
        );"""

sql_create_1stscrape_table = """
        CREATE TABLE IF NOT EXISTS firstScrape(
        Company text NOT NULL,
        Website text,
        Scraped_text text,
        Links_1st text,
        Company_id integer NOT NULL
        );"""

["Company", "ID_link", "URL", "Source_code", "Text"]

sql_create_2ndscrape_table = """
        CREATE TABLE IF NOT EXISTS secondScrape(
        Company text NOT NULL,
        ID_link integer NOT NULL,
        URL text,
        Source_code text,
        Text text
        );"""

sql_create_topics = """
        CREATE TABLE IF NOT EXISTS topics(
        index_topic int,
        topic text
        );"""

def create_tables():
        global database
                        
        # create a database connection
        conn = create_connection(database)

        if conn is not None:
            # create project table
            create_table(conn, sql_create_companies_table)
            create_table(conn, sql_create_1stscrape_table)
            create_table(conn, sql_create_2ndscrape_table)
#             create_table(conn, sql_create_topics)
        
            num_topics_int = 10

            for num in range(num_topics_int):
                name = "Score_" + str(num)
                modify_table(conn, ("ALTER TABLE secondScrape ADD " + name + " integer"))
                        
            print("all succeeded")

        else:

            print(" error! cannot create the database connection")  

In [ ]:
create_tables()

## Step1C: Reading the CSV file and writing to the SQLite file


In [ ]:
df_CSV = read_CSV("191005_PilotGroup.csv")
table = "companies"
conn = create_connection(database)
write_to_db(df_CSV, table, conn)
conn.close()
    

# Execute scraping

In [ ]:
def scrape_sites():
    table = "companies"
    sql = "SELECT * FROM " + table
    
    conn = create_connection(database)
    # Step 2A
    df_data = retrieve_data(sql, conn)
    
    # Step 2B
    for index, row in df_data.iterrows():
        id_link = index
        company = row['Company']
        url = row['Website']
        try:
            res_text = scrape(url)
        except:
            res_text = "No response"
                
        # Step 2C
        soup = bs4.BeautifulSoup(res_text, 'lxml')
        links = []
        for link in soup.find_all('a', href=True):
            print(link['href'])
            links.append(link['href'])
        links_1st = []
        for link in links:
            if "https" in link and not url in link:
                next
            elif link == url:
                next
            else:
                links_1st.append(link)
        links_1st = str(list(set(links_1st)))
        links_1st = links_1st.replace("'", "")
        links_1st = links_1st.replace('"', "")
        links_1st = links_1st.replace(" ", "")  
        write_to_db_scrape1(company, url, res_text, links_1st, id_link, conn)    
    conn.close()

In [ ]:
scrape_sites()